In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tqdm import tqdm
import time
import cv2
import numpy as np
import pandas as pd
import os
# os.environ["CUDA_VISIBLE_DEVICES"]="1"; 

import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation,Dropout,Conv2D, MaxPooling2D,BatchNormalization
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.models import Model

In [ ]:
df_val=pd.read_csv('../input/data-csv/df_val.csv')
df_train=pd.read_csv('../input/data-csv/df_train.csv')
df_test=pd.read_csv('../input/data-csv/df_test.csv')

In [ ]:
Image_Width=71
Image_Height=71
Image_Size=(Image_Width,Image_Height)
Image_Channels=3
batch_size=128

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,width_shift_range=0.1, height_shift_range=0.1, fill_mode="nearest")
train_generator = train_datagen.flow_from_dataframe(df_train, directory="../input/cifar10-object-recognition-in-images-zip-file/train_test/train/train/",x_col='id',y_col='label', target_size=Image_Size, class_mode='categorical', batch_size=batch_size,)

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(df_val,  "../input/cifar10-object-recognition-in-images-zip-file/train_test/train/train/",x_col='id',y_col='label',shuffle=False,target_size=Image_Size, class_mode='categorical', batch_size=batch_size )

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(df_test,  "../input/cifar10-object-recognition-in-images-zip-file/train_test/train/train/",x_col='id',y_col='label',shuffle=False,target_size=Image_Size, class_mode='categorical', batch_size=batch_size )


# Xception

In [ ]:

# Build model
base_model = tf.keras.applications.xception.Xception(include_top=False,
                                                     weights='imagenet',
                                                     input_shape = (71,71, 3),
                                                     pooling = "max"
                                                    )

base_model.trainable = True

x = base_model.output
x = BatchNormalization() (x)
x = Dense(64, activation = "relu") (x)
x = Dropout(0.3) (x)
outputs = Dense(10, activation = "softmax") (x)

model = Model(inputs = base_model.input, outputs = outputs)
lr = 0.0001

model.compile(optimizer = Adamax(learning_rate = lr), loss = "categorical_crossentropy", metrics = ["accuracy"])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
ckpoint = ModelCheckpoint("kaggle/working/best_weights_xception_loss_model.h5", monitor="val_loss", save_best_only=True, mode="auto")

In [ ]:
n_epochs = 20
history=model.fit(x = train_generator, 
         epochs = n_epochs,
         validation_data = validation_generator,
         callbacks = [ckpoint])

In [ ]:
H=history
# PLOT MODEL HISTORY OF ACCURACY AND LOSS OVER EPOCHS
plt.plot(H.history['accuracy'])
plt.plot(H.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.savefig('Initial_Xception_Model_Accuracy.png')
plt.show()
# summarize history for loss
plt.plot(H.history['loss'])
plt.plot(H.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig('Initial_Xception_Model_loss.png')
plt.show()

# predict

In [ ]:
model=tf.keras.models.load_model ('../input/data-csv/best_weights_xception_loss_model.h5')

In [ ]:
predict = model.predict(test_generator)

In [ ]:
df_test['actual_lable'] =  test_generator.classes
df_test['predict'] = np.argmax(predict, axis=-1)
data_label=[]
for k,v in test_generator.class_indices.items():
  data_label.append(str(k))

In [ ]:

from sklearn.metrics import classification_report
print(classification_report(df_test['actual_lable'], df_test['predict'], target_names = data_label))


In [ ]:
# CREATE CONFUSION MATRIX OF ACTUAL VS. PREDICTION 
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(df_test['actual_lable'], df_test['predict'])
plt.figure(figsize = (12, 10))
cm = pd.DataFrame(cm , index = data_label , columns = data_label)
import seaborn as sns
ax = sns.heatmap(cm, linecolor='white', cmap='Blues', linewidth=1, annot=True, fmt='')
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)
plt.title('Confusion Matrix', size=20)
plt.xlabel('Predicted Labels', size=14)
plt.ylabel('Actual Labels', size=14)
plt.savefig('Initial_Xception_Model_Confusion_Matrix.png')
plt.show()

In [ ]:
a=[]
count=0
for i in range(len(df_test)):
    if df_test['actual_lable'][i]== df_test['predict'][i]:
        a.append(np.max(predict[i],axis=-1))
        if np.max(predict[i],axis=-1) <0.6:
            count+=1
#             print(df_test['id'][i])
#             print(df_test['label'][i])
#             print(df_test['predict'][i])
print(len(a))
print(count)
print(np.min(a))

# for i in range(len(df_wrong_val)):
#     if df_wrong_val['actual_lable'][i]!= df_wrong_val['predict'][i]:
#         a=np.max(predict[i])
# print(np.max(a))

In [ ]:
data_train_addition_id=[]
data_train_addition_label=[]
for i in tqdm(range(len(df_test))):
    if (df_test['actual_lable'][i] !=df_test['predict'][i]):
        data_train_addition_id.append(df_test['id'][i])
        data_train_addition_label.append(df_test['label'][i])
df_aug=pd.DataFrame()
df_aug['id']=pd.DataFrame(data_train_addition_id)
df_aug['label']=pd.DataFrame(data_train_addition_label)
df_aug.to_csv('wrong_test.csv',index=False)

In [ ]:
predict = model.predict(validation_generator)

In [ ]:
df_val['actual_lable'] =  validation_generator.classes
df_val['predict'] = np.argmax(predict, axis=-1)
data_label=[]
for k,v in validation_generator.class_indices.items():
  data_label.append(str(k))

In [ ]:

from sklearn.metrics import classification_report
print(classification_report(df_val['actual_lable'], df_val['predict'], target_names = data_label))


In [ ]:
data_train_addition_id=[]
data_train_addition_label=[]
for i in tqdm(range(len(df_val))):
    if (df_val['actual_lable'][i] !=df_val['predict'][i]):
        data_train_addition_id.append(df_val['id'][i])
        data_train_addition_label.append(df_val['label'][i])
df_aug=pd.DataFrame()
df_aug['id']=pd.DataFrame(data_train_addition_id)
df_aug['label']=pd.DataFrame(data_train_addition_label)
df_aug.to_csv('wrong_val.csv',index=False)

Predict Train

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_dataframe(df_train, directory="../input/cifar10-object-recognition-in-images-zip-file/train_test/train/train/",shuffle=False,x_col='id',y_col='label', target_size=Image_Size, class_mode='categorical', batch_size=batch_size,)

predict = model.predict(train_generator)

In [ ]:
df_train['actual_lable'] =  train_generator.classes
df_train['predict'] = np.argmax(predict, axis=-1)
data_label=[]
for k,v in train_generator.class_indices.items():
  data_label.append(str(k))

In [ ]:

from sklearn.metrics import classification_report
print(classification_report(df_train['actual_lable'], df_train['predict'], target_names = data_label))


In [ ]:
# CREATE CONFUSION MATRIX OF ACTUAL VS. PREDICTION 
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(df_train['actual_lable'], df_train['predict'])
plt.figure(figsize = (12, 10))
cm = pd.DataFrame(cm , index = data_label , columns = data_label)
import seaborn as sns
ax = sns.heatmap(cm, linecolor='white', cmap='Blues', linewidth=1, annot=True, fmt='')
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)
plt.title('Confusion Matrix', size=20)
plt.xlabel('Predicted Labels', size=14)
plt.ylabel('Actual Labels', size=14)
plt.savefig('Initial_Train_Xception_Model_Confusion_Matrix.png')
plt.show()

In [ ]:
df_train.to_csv('df_train_add.csv',index=False)
df_test.to_csv('df_test_add.csv',index=False)

In [ ]:
df_test

In [ ]:
df_train=pd.read_csv('df_train_add.csv')
df_test=pd.read_csv('df_test_add.csv')

In [ ]:
for i in tqdm(range(len(df_test))):
    if (df_test['actual_lable'][i] ==1) and (df_test['predict'][i] ==9):
        fname=df_test['id'][i]
        print(fname)
        img=cv2.imread(os.path.join('../input/cifar10-object-recognition-in-images-zip-file/train_test/train/train/',fname))
        imgplot = plt.imshow(img)
        plt.show() 

In [ ]:
count=0
for k in range(0,10):
    for i in tqdm(range(len(df_test))):
        if (df_test['actual_lable'][i] !=df_test['predict'][i]):
            if (df_test['actual_lable'][i]==k):
                count+=1
    print(k,count)
    count=0

count=0
print('TRAINING')
for k in range(0,10):
    for i in tqdm(range(len(df_train))):
        if (df_train['actual_lable'][i] !=df_train['predict'][i]):
            if (df_train['actual_lable'][i]==k):
                count+=1
    print(k,count)
    count=0

In [ ]:
data_train_addition_id=[]
data_train_addition_label=[]
for i in tqdm(range(len(df_train))):
    if (df_train['actual_lable'][i] !=df_train['predict'][i]):
        data_train_addition_id.append(df_train['id'][i])
        data_train_addition_label.append(df_train['label'][i])
df_aug=pd.DataFrame()
df_aug['id']=pd.DataFrame(data_train_addition_id)
df_aug['label']=pd.DataFrame(data_train_addition_label)
df_aug.to_csv('wrong_train.csv',index=False)

In [ ]:
len(df_aug)

# Data Augmentation

In [ ]:
!pip install py7zr
import py7zr

In [ ]:
archive=py7zr.SevenZipFile('../input/cifar-10/train.7z', mode='r')
archive.extractall(path="kaggle/working/temp")
archive.close()

In [ ]:
df_wrong_train=pd.read_csv('../input/data-csv/wrong_train.csv')
df_wrong_val=pd.read_csv('./wrong_val.csv')
df_wrong_test=pd.read_csv('./wrong_test.csv')

In [ ]:
# import os
# len(os.listdir('kaggle/working/temp/train/'))

In [ ]:
data_train_addition_id=[]
data_train_addition_label=[]

In [ ]:
# df_wrong_train

In [ ]:
path_dir_save='kaggle/working/temp/train/'
from tqdm import tqdm
for i in tqdm(range(len(df_wrong_train))):
    fname=df_wrong_train['id'][i]
    img=cv2.imread(os.path.join('kaggle/working/temp/train/',fname))
    rotated_img=cv2.flip(img, 1)
    rotated_img_name=fname[:(len(fname)-4)] + '_flip_'+fname[(len(fname)-4):]
    data_train_addition_id.append(rotated_img_name)
    data_train_addition_label.append(df_wrong_train['label'][i])
    cv2.imwrite(os.path.join(path_dir_save, rotated_img_name),rotated_img)

In [ ]:
df_wrong_train2=pd.DataFrame()
df_wrong_train2['id']=pd.DataFrame(data_train_addition_id)
df_wrong_train2['label']=pd.DataFrame(data_train_addition_label)
df_wrong_train=pd.concat([df_wrong_train,df_wrong_train2],ignore_index=True)

In [ ]:
data_train_addition_id=[]
data_train_addition_label=[]

In [ ]:
goc=10
path_dir_save='kaggle/working/temp/train/'
for i in tqdm(range(len(df_wrong_train))):
    fname=str(df_wrong_train['id'][i])
#     print(len(fname))
#     print(os.path.join('kaggle/working/temp3/train/',fname))
    img=cv2.imread(os.path.join('kaggle/working/temp/train/',fname))
    h,w=img.shape[:2]
    rotation_matrix=cv2.getRotationMatrix2D((w/2,h/2),goc,.8)
    rotated_img=cv2.warpAffine(img,rotation_matrix,(w,h))
    rotated_img_name=fname[:(len(fname)-4)] + '_rotated_'+str(goc)+fname[(len(fname)-4):]
    data_train_addition_id.append(rotated_img_name)
    data_train_addition_label.append(df_wrong_train['label'][i])
    cv2.imwrite(os.path.join(path_dir_save, rotated_img_name),rotated_img)

In [ ]:
df_wrong_train2=pd.DataFrame()
df_wrong_train2['id']=pd.DataFrame(data_train_addition_id)
df_wrong_train2['label']=pd.DataFrame(data_train_addition_label)
df_wrong_train=pd.concat([df_wrong_train,df_wrong_train2],ignore_index=True)

In [ ]:
len(df_wrong_train)

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,width_shift_range=0.1, height_shift_range=0.1, fill_mode="nearest")
train_generator = train_datagen.flow_from_dataframe(df_wrong_train, directory="kaggle/working/temp/train/",x_col='id',y_col='label', target_size=Image_Size, class_mode='categorical', batch_size=batch_size,)

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(df_wrong_val,  "kaggle/working/temp/train/",x_col='id',y_col='label',target_size=Image_Size, class_mode='categorical', batch_size=batch_size )

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(df_wrong_test,  "kaggle/working/temp/train/",x_col='id',y_col='label',shuffle=False,target_size=Image_Size, class_mode='categorical', batch_size=batch_size )


# # Xception with df_aug

In [ ]:

# Build model
base_model = tf.keras.applications.xception.Xception(include_top=False,
                                                     weights='imagenet',
                                                     input_shape = (71,71, 3),
                                                     pooling = "max"
                                                    )

base_model.trainable = True

x = base_model.output
x = BatchNormalization() (x)
x = Dense(32, activation = "relu") (x)
x = Dropout(0.3) (x)
outputs = Dense(10, activation = "softmax") (x)

model = Model(inputs = base_model.input, outputs = outputs)
lr = 0.00001

model.compile(optimizer = Adamax(learning_rate = lr), loss = "categorical_crossentropy", metrics = ["accuracy"])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
ckpoint = ModelCheckpoint("best_weights_xception_wrong_aug_loss_model.h5", monitor="val_loss", save_best_only=True, mode="auto")
ckpoint2 = ModelCheckpoint("best_weights_xception_wrong_aug_acc_model.h5", monitor="val_accuracy", save_best_only=True, mode="auto")
earlystop = EarlyStopping(monitor="val_loss",
                          min_delta=0,
                          mode="auto",
                          patience=100,
                          restore_best_weights=True,)
earlystop2 = EarlyStopping(monitor="val_accuracy",
                          min_delta=0,
                          mode="auto",
                          patience=20,
                          restore_best_weights=True,)

In [ ]:
n_epochs = 300
history=model.fit(x = train_generator, 
         epochs = n_epochs,
         validation_data = validation_generator,
         callbacks = [ckpoint,ckpoint2,earlystop2])

In [ ]:
H=history
# PLOT MODEL HISTORY OF ACCURACY AND LOSS OVER EPOCHS
plt.plot(H.history['accuracy'])
plt.plot(H.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.savefig('Initial_Xception_aug_Model_Accuracy.png')
plt.show()
# summarize history for loss
plt.plot(H.history['loss'])
plt.plot(H.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig('Initial_Xception_aug_Model_loss.png')
plt.show()

# CNN with df_aug

In [ ]:
Image_Width=71
Image_Height=71
Image_Channels=3

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,\
     Dropout,Flatten,Dense,Activation,\
     BatchNormalization

model=Sequential()

model.add(Conv2D(16,(3,3),activation='relu',input_shape=(Image_Width,Image_Height,Image_Channels)))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Conv2D(32,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))


model.add(Conv2D(128,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Conv2D(256,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(10,activation='softmax'))
opt = keras.optimizers.Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy'])
model.summary()

In [ ]:
from keras.callbacks import LearningRateScheduler, ModelCheckpoint,EarlyStopping

checkpoint=ModelCheckpoint('model_CNN_acc.h5',
                           monitor="val_accuracy",
                           verbose=0,
                           save_best_only=True,
                           save_weights_only=False,
                           mode="auto",
                           save_freq="epoch")
checkpoint2=ModelCheckpoint('model_CNN_loss.h5',
                           monitor="val_loss",
                           verbose=0,
                           save_best_only=True,
                           save_weights_only=False,
                           mode="auto",
                           save_freq="epoch")


def step_decay(epoch):
    # initialize the base initial learning rate, drop factor, and
    # epochs to drop every
    initAlpha = 0.01
    decay_rate = 0.005

    # compute learning rate for the current epoch
    alpha = initAlpha * 1/(1 + decay_rate * epoch)

    # return the learning rate
    return float(alpha)
earlystop = EarlyStopping(monitor="val_accuracy",
                          min_delta=0,
                          mode="auto",
                          patience=20,
                          restore_best_weights=True,)
callbacks = [LearningRateScheduler(step_decay),checkpoint,checkpoint2,earlystop]

In [ ]:
n_epochs = 500
history=model.fit(x = train_generator, 
         epochs = n_epochs,
         validation_data = validation_generator,
         callbacks = callbacks)

In [ ]:
H=history
# PLOT MODEL HISTORY OF ACCURACY AND LOSS OVER EPOCHS
plt.plot(H.history['accuracy'])
plt.plot(H.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.savefig('Initial_Xception_aug_Model_Accuracy.png')
plt.show()
# summarize history for loss
plt.plot(H.history['loss'])
plt.plot(H.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig('Initial_Xception_aug_Model_loss.png')
plt.show()

In [ ]:
model=tf.keras.models.load_model ('./best_weights_xception_wrong_aug_loss_model.h5')
# predict = model.predict(test_generator)
predict = model.predict(validation_generator)

In [ ]:
# file1 = open("predict_wrong_test.txt", "w") 
# for i in range(len(predict)):
#     file1.write(str(predict[i]))
# file1.close()


In [ ]:
len(validation_generator)

In [ ]:
# df_wrong_test['actual_lable'] =  test_generator.classes
# df_wrong_test['predict'] = np.argmax(predict, axis=-1)
# data_label=[]
# for k,v in test_generator.class_indices.items():
#   data_label.append(str(k))

df_wrong_val['actual_lable'] =  validation_generator.classes
df_wrong_val['predict'] = np.argmax(predict, axis=-1)
data_label=[]
for k,v in validation_generator.class_indices.items():
  data_label.append(str(k))

In [ ]:

from sklearn.metrics import classification_report
print(classification_report(df_wrong_test['actual_lable'], df_wrong_test['predict'], target_names = data_label))


In [ ]:
# # CREATE CONFUSION MATRIX OF ACTUAL VS. PREDICTION 
# from sklearn.metrics import confusion_matrix
# cm = confusion_matrix(df_wrong_test['actual_lable'], df_wrong_test['predict'])
# plt.figure(figsize = (12, 10))
# cm = pd.DataFrame(cm , index = data_label , columns = data_label)
# import seaborn as sns
# ax = sns.heatmap(cm, linecolor='white', cmap='Blues', linewidth=1, annot=True, fmt='')
# bottom, top = ax.get_ylim()
# ax.set_ylim(bottom + 0.5, top - 0.5)
# plt.title('Confusion Matrix', size=20)
# plt.xlabel('Predicted Labels', size=14)
# plt.ylabel('Actual Labels', size=14)
# plt.savefig('Initial_Xception_aug_acc_Model_Confusion_Matrix.png')
# plt.show()

# CREATE CONFUSION MATRIX OF ACTUAL VS. PREDICTION 
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(df_wrong_val['actual_lable'], df_wrong_val['predict'])
plt.figure(figsize = (12, 10))
cm = pd.DataFrame(cm , index = data_label , columns = data_label)
import seaborn as sns
ax = sns.heatmap(cm, linecolor='white', cmap='Blues', linewidth=1, annot=True, fmt='')
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)
plt.title('Confusion Matrix', size=20)
plt.xlabel('Predicted Labels', size=14)
plt.ylabel('Actual Labels', size=14)
plt.savefig('Initial_Xception_aug_acc_Model_Confusion_Matrix.png')
plt.show()

In [ ]:
# for i in range(len(df_wrong_test)):
#     if df_wrong_test['actual_lable'][i]!= df_wrong_test['predict'][i]:
#         a=np.max(predict[i])
# print(np.max(a))

for i in range(len(df_wrong_val)):
    if df_wrong_val['actual_lable'][i]!= df_wrong_val['predict'][i]:
        a=np.max(predict[i])
print(np.max(a))

# EfficentnetB0

In [ ]:

# Build model
base_model = tf.keras.applications.efficientnet.EfficientNetB0(include_top=False,
                                                     weights='imagenet',
                                                     input_shape = (71,71, 3),
                                                     pooling = "max"
                                                    )

base_model.trainable = True

x = base_model.output
x = BatchNormalization() (x)
x = Dense(64, activation = "relu") (x)
x = Dropout(0.3) (x)
outputs = Dense(10, activation = "softmax") (x)

model = Model(inputs = base_model.input, outputs = outputs)
lr = 0.0001

model.compile(optimizer = Adamax(learning_rate = lr), loss = "categorical_crossentropy", metrics = ["accuracy"])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
ckpoint = ModelCheckpoint("kaggle/working/best_weights_efficentnetB0_model_.h5", monitor="val_accuracy", save_best_only=True, mode="auto")

In [ ]:
n_epochs = 20
history=model.fit(x = train_generator, 
         epochs = n_epochs,
         validation_data = validation_generator,
         callbacks = [ckpoint])

# MobileNet

In [ ]:

# Build model
base_model = tf.keras.applications.MobileNet(include_top=False,
                                                                 weights='imagenet',
                                                                 input_shape = (71,71, 3),
                                                                 pooling = "max"
                                                                )

base_model.trainable = True

x = base_model.output
x = BatchNormalization() (x)
x = Dense(64, activation = "relu") (x)
x = Dropout(0.3) (x)
outputs = Dense(10, activation = "softmax") (x)

model = Model(inputs = base_model.input, outputs = outputs)
lr = 0.0001

model.compile(optimizer = Adamax(learning_rate = lr), loss = "categorical_crossentropy", metrics = ["accuracy"])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
ckpoint = ModelCheckpoint("kaggle/working/best_weights_mobilenet_model_.h5", monitor="val_accuracy", save_best_only=True, mode="auto")

In [ ]:
n_epochs = 20
history=model.fit(x = train_generator, 
         epochs = n_epochs,
         validation_data = validation_generator,
         callbacks = [ckpoint])

# VGG19

In [ ]:

# Build model
base_model = tf.keras.applications.vgg19.VGG19(include_top=False,
                                                                 weights='imagenet',
                                                                 input_shape = (71,71, 3),
                                                                 pooling = "max"
                                                                )

base_model.trainable = True

x = base_model.output
x = BatchNormalization() (x)
x = Dense(64, activation = "relu") (x)
x = Dropout(0.3) (x)
outputs = Dense(10, activation = "softmax") (x)

model = Model(inputs = base_model.input, outputs = outputs)
lr = 0.0001

model.compile(optimizer = Adamax(learning_rate = lr), loss = "categorical_crossentropy", metrics = ["accuracy"])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
ckpoint = ModelCheckpoint("kaggle/working/best_weights_vgg19_model_.h5", monitor="val_accuracy", save_best_only=True, mode="auto")

In [ ]:
n_epochs = 20
history=model.fit(x = train_generator, 
         epochs = n_epochs,
         validation_data = validation_generator,
         callbacks = [ckpoint])

# Resnet50

In [ ]:

# Build model
base_model = tf.keras.applications.resnet50.ResNet50.(include_top=False,
                                                                 weights='imagenet',
                                                                 input_shape = (71,71, 3),
                                                                 pooling = "max"
                                                                )

base_model.trainable = True

x = base_model.output
x = BatchNormalization() (x)
x = Dense(64, activation = "relu") (x)
x = Dropout(0.3) (x)
outputs = Dense(10, activation = "softmax") (x)

model = Model(inputs = base_model.input, outputs = outputs)
lr = 0.0001

model.compile(optimizer = Adamax(learning_rate = lr), loss = "categorical_crossentropy", metrics = ["accuracy"])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
ckpoint = ModelCheckpoint("kaggle/working/best_weights_resnet50_model_.h5", monitor="val_accuracy", save_best_only=True, mode="auto")

In [ ]:
n_epochs = 20
history=model.fit(x = train_generator, 
         epochs = n_epochs,
         validation_data = validation_generator,
         callbacks = [ckpoint])